In [2]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import math
import time
import pyproj
import random
#import rasterio
#from rasterio import features

In [3]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)


df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

source = ogr.Open(gdf.to_json())

#[['geometry','value']].

wkt_geom = [i.wkt for i in gdf.geometry]
#gdf.explore()

In [4]:
gdf

,Unnamed: 0,geometry,cor,application_rate,value,cultural_practice,bin_element_numbers,start_date,end_date,type
0,0,"LINESTRING (-89.77969 40.02795, -89.77969 40.0...",produzindo,NaN,6.661111,0,0,2022-09-28 23:15:07.700,2022-09-28 23:15:43.700,RASTRO
1,1,"LINESTRING (-89.77961 40.02747, -89.77961 40.0...",produzindo,NaN,6.302778,0,0,2022-09-28 23:16:04.700,2022-09-28 23:16:41.700,RASTRO
2,2,"LINESTRING (-89.77963 40.02807, -89.77963 40.0...",produzindo,NaN,7.022764,0,0,2022-09-28 23:16:43.700,2022-09-28 23:18:46.700,RASTRO
3,3,"LINESTRING (-89.77959 40.03013, -89.77959 40.0...",produzindo,NaN,6.549242,0,0,2022-09-28 23:21:15.700,2022-09-28 23:23:27.700,RASTRO
4,4,"LINESTRING (-89.77955 40.02793, -89.77955 40.0...",produzindo,NaN,7.026667,0,0,2022-09-28 23:23:31.700,2022-09-28 23:24:01.700,RASTRO
...,...,...,...,...,...,...,...,...,...,...
487,487,"POLYGON ((-89.77926 40.02972, -89.77926 40.029...",humidity_rastro,NaN,22.030000,0,0,2022-09-28 23:41:48.000,2022-09-28 23:41:56.000,HUMIDITY
488,488,"POLYGON ((-89.77926 40.02986, -89.77926 40.029...",humidity_rastro,NaN,21.524444,0,0,2022-09-28 23:41:56.000,2022-09-28 23:42:04.000,HUMIDITY
489,489,"POLYGON ((-89.77925 40.03000, -89.77918 40.029...",humidity_rastro,NaN,15.000000,0,0,2022-09-28 23:42:04.000,2022-09-28 23:42:06.000,HUMIDITY
490,490,"POLYGON ((-89.77912 40.02991, -89.77912 40.029...",humidity_rastro,NaN,15.000000,0,0,2022-09-28 23:44:50.000,2022-09-28 23:45:01.000,HUMIDITY


In [88]:
source_ds = ogr.GetDriverByName("Memory").CopyDataSource(source, "")
source_layer = source_ds.GetLayer(0)
source_srs = source_layer.GetSpatialRef()
x_min, x_max, y_min, y_max = source_layer.GetExtent()
numFeatures = source_layer.GetFeatureCount()



#random 
new_field = ogr.FieldDefn('feat1', ogr.OFTReal)
source_layer.CreateField(new_field)
source_layer_def = source_layer.GetLayerDefn()
fd_id = source_layer_def.GetFieldIndex('value')
ft_id1 = source_layer_def.GetFieldIndex('feat1')

#random 
new_field = ogr.FieldDefn('feat2', ogr.OFTReal)
source_layer.CreateField(new_field)
source_layer_def = source_layer.GetLayerDefn()
fd_id = source_layer_def.GetFieldIndex('value')
ft_id2 = source_layer_def.GetFieldIndex('feat2')


ldefn = source_layer.GetLayerDefn()
ldefn.GetFieldCount()
lsAtribNames = [field.name for field in source_layer.schema]

In [95]:
lsAtribNames

['id',
 'Unnamed: 0',
 'cor',
 'application_rate',
 'value',
 'cultural_practice',
 'bin_element_numbers',
 'start_date',
 'end_date',
 'type',
 'feat1',
 'feat2']

In [83]:
def map_val(x):
    return (x - min(gdf.value)) * (0xff - 0) / (max(gdf.value) - min(gdf.value))

In [90]:
map_val(gdf.value.loc[0])

71.36649417136381

In [92]:
map_val = lambda x : (x - min(gdf.value)) * (0xff - 0) / (max(gdf.value) - min(gdf.value))

In [93]:
map_val(gdf.value.loc[0])

71.36649417136381

In [94]:
source_layer[0].GetField(fd_id), source_layer[0].GetField(ft_id2)

(6.661111111111111, None)

In [89]:
map_val(source_layer[0].GetField(fd_id))

71.36649417136381

In [97]:
for feature in source_layer:
    feature.SetField(ft_id1, random.randint(0, 0xFF))
    source_layer.SetFeature(feature)

In [98]:
for feature in source_layer:
    feature.SetField(ft_id2, map_val(feature.GetField(fd_id)))
    source_layer.SetFeature(feature)

In [11]:
pix_size = 0.000002
x_res = int((x_max - x_min) / pix_size)
y_res = int((y_max - y_min) / pix_size)

In [102]:
target_ds = gdal.GetDriverByName('GTiff').Create('rasterize1.tif', x_res, y_res, 3, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pix_size, 0, y_max, 0, -pix_size))

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'

if source_srs:
    target_ds.SetProjection(source_srs.ExportToWkt())
else:
    target_ds.SetProjection(proj)

err = gdal.RasterizeLayer(target_ds, (3, 2, 1), source_layer,
    burn_values=(0xff, 0xff, 0xff),
    options=["ATTRIBUTE=%s" % 'feat1'])


pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize1.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E9251A4420> >

In [101]:
target_ds = gdal.GetDriverByName('GTiff').Create('rasterize2.tif', x_res, y_res, 3, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_min, pix_size, 0, y_max, 0, -pix_size))

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'

if source_srs:
    target_ds.SetProjection(source_srs.ExportToWkt())
else:
    target_ds.SetProjection(proj)

err = gdal.RasterizeLayer(target_ds, (3, 2, 1), source_layer,
    burn_values=(0xff, 0xff, 0xff),
    options=["ATTRIBUTE=%s" % 'feat2'])


pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize2.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E923AE7930> >

In [76]:
x_size = gdf.total_bounds[2]-gdf.total_bounds[0]
y_size = gdf.total_bounds[3]-gdf.total_bounds[1]
x_0 = gdf.total_bounds[0]
y_0 = gdf.total_bounds[3]

 

pix_size = 0.000002
pixels_in_x = math.ceil(x_size/pix_size)
pixels_in_y = math.ceil(y_size/pix_size)

 
sr_wkt = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
sr = osr.SpatialReference( sr_wkt )


# Create a memory raster to rasterize into.

 

target_ds = gdal.GetDriverByName('MEM').Create('', pixels_in_x, pixels_in_y, 4, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))

 

# Create a memory layer to rasterize from.

 
rast_ogr_ds  = ogr.GetDriverByName('Memory').CreateDataSource('wrk')
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr)


for g in gdf[gdf['cor'] == 'produzindo_rastro'].geometry:
    feat = ogr.Feature(rast_mem_lyr.GetLayerDefn())
    feat.SetGeometryDirectly(ogr.Geometry(wkt = g.wkt))
    rast_mem_lyr.CreateFeature(feat)

gdal.RasterizeLayer(target_ds, [4,3,2,1], rast_mem_lyr, burn_values=(0, 0xff, 0xff, 0 ))

""" , options = ["BURN_VALUE_FROM="]   ,  ,options=['ATTRIBUTE=values']  burn_values=(0xFFFF, 0xFFFF, 0xFFFF, 0 )"""

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
target_ds.SetProjection(proj)

 

tiffdriver = gdal.GetDriverByName('GTiff')
tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001E923DCFD80> >